In [1]:
# Run this only once
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("semcor") # downloads the .zip file, but doesn't unzip it
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package punkt to /home/sainath/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sainath/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package semcor to /home/sainath/nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sainath/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sainath/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
from gensim.models.keyedvectors import KeyedVectors
W2V = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary = True)

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [9]:
import nltk
from nltk import word_tokenize
from nltk.corpus import semcor # corpus reader: https://github.com/nltk/nltk/blob/develop/nltk/corpus/reader/semcor.py
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag

import random
import numpy as np
from tqdm.notebook import tqdm
from string import punctuation
from num2words import num2words
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [70]:
lemmatizer = WordNetLemmatizer()

def cosineSimilarity(a, b):
    cs = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    return cs

def isNumber(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def n2w(w):
    # converts given n to word form if n is numeric
    if isNumber(w) and w.lower() != "infinity" and w.lower() != "nan":
        w = num2words(w)
    return w

def lemmatize(w, tag):
    if tag is None:
        return lemmatizer.lemmatize(w)
    else:
        return lemmatizer.lemmatize(w, tag)

# def clean(tokens):
#     tagged = nltk.pos_tag(tokens)
#     lemmatized = [lemmatize(w, treebank2wn(tag)) for w, tag in tagged]
#     cleaned = [n2w(w) for w in lemmatized if w.lower() not in SW]
#     return cleaned

In [10]:
x = pos_tag(word_tokenize("This is a doll"),tagset='universal')
x

[('This', 'DET'), ('is', 'VERB'), ('a', 'DET'), ('doll', 'NOUN')]

In [11]:
#Only for nouns
W2V.word_vec("Hello")

# print(type(W2V.word_vec("Hello")))

/tmp/ipykernel_107127/1565140508.py:2: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  W2V.word_vec("Hello")


array([-0.05102539,  0.12060547, -0.01257324,  0.03088379, -0.08544922,
        0.0534668 , -0.13964844, -0.16308594,  0.08300781,  0.20507812,
       -0.09423828,  0.15527344, -0.00418091,  0.02111816, -0.10986328,
        0.22460938,  0.22265625,  0.15917969,  0.05786133, -0.15527344,
        0.23046875,  0.34765625,  0.44335938, -0.14550781,  0.15136719,
        0.02209473, -0.16308594,  0.2578125 ,  0.20019531,  0.06054688,
       -0.0222168 , -0.13183594, -0.06396484, -0.12792969,  0.09814453,
       -0.13574219,  0.01928711,  0.25195312,  0.14257812,  0.2421875 ,
        0.03173828, -0.17089844,  0.26171875,  0.4296875 ,  0.24804688,
       -0.10351562, -0.13867188, -0.14257812, -0.20703125, -0.00860596,
       -0.43359375, -0.01757812,  0.40234375,  0.32617188,  0.33203125,
        0.01000977, -0.11523438, -0.12695312,  0.01940918, -0.25390625,
        0.00393677, -0.00344849, -0.19042969,  0.05419922,  0.14453125,
       -0.0546875 , -0.11132812,  0.15136719, -0.265625  ,  0.20

In [12]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/sasank/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [39]:
wn.synsets("bank",'n')

[Synset('bank.n.01'),
 Synset('depository_financial_institution.n.01'),
 Synset('bank.n.03'),
 Synset('bank.n.04'),
 Synset('bank.n.05'),
 Synset('bank.n.06'),
 Synset('bank.n.07'),
 Synset('savings_bank.n.02'),
 Synset('bank.n.09'),
 Synset('bank.n.10')]

STATEMENT FOR INPUT

In [99]:
statement = "Little John was looking for his toy box"
statement = "The river is flooding on the bank"

pos_tagged_statement =  pos_tag(word_tokenize(statement),tagset='universal')
statement = statement.split()

In [100]:
pos_tagged_statement

[('The', 'DET'),
 ('river', 'NOUN'),
 ('is', 'VERB'),
 ('flooding', 'VERB'),
 ('on', 'ADP'),
 ('the', 'DET'),
 ('bank', 'NOUN')]

In [101]:
def context_embed(statement,word):
    context_embedding = np.zeros(300)
    for i in statement:
        if i!=word:
            try:
                context_embedding = context_embedding + W2V.word_vec(i)
            except:
                continue
    statemen_len = len(statement)
    return context_embedding/(statemen_len-1)

In [102]:
import re

def sense_embed(synset_id):
    sense_embedding = np.zeros(300)
    text = synset_id.definition()
    re.sub(r"[^-9A-Za-z ]", "" , text)
    text = text.replace("(","")
    text = text.replace(")","")
    text = text.replace(";","")
    text = text.split()
    sense_embedding = context_embed(text,"")
    # print(text)
    return sense_embedding

In [103]:
words_sense_noun = []

for i in pos_tagged_statement:
    tag = i[1]
    word = i[0]
    if(tag != "NOUN"):
        continue
    # words_sense_noun.append(word)
    cont_embed = context_embed(statement,i)
    sen_embed = []
    poss_senses = wn.synsets(word,'n')
    # print(poss_senses)
    for i in poss_senses:
        text_hyper = i.hypernyms()
        text_hypo = i.hyponyms()
        # print("Hyper :",text_hyper)
        # print("Hypo :",text_hypo)
        # print(i.definition())
        x = sense_embed(i)
        for j in text_hypo:
            x += sense_embed(j)
        for j in text_hyper:
            x += sense_embed(j)
        sen_embed.append(x/(1+len(text_hyper)+len(text_hypo)))
    cosine_similarity = []
    for i in sen_embed:
        cosine_similarity.append(cosineSimilarity(cont_embed,i))
    max_similar_sense = poss_senses[np.argmax(cosine_similarity)]
    print(word," : ",max_similar_sense.definition())

river  :  a large natural stream of water (larger than a creek)
bank  :  sloping land (especially the slope beside a body of water)


/tmp/ipykernel_107127/1121871512.py:6: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  context_embedding = context_embedding + W2V.word_vec(i)


In [112]:
data = semcor.tagged_sents(tag = "sem")

In [132]:
data[0][2]#???? What to do here showing error for data[0][1,2,4..]
#Please look into this and also how to extract nouns from this and all ....

ModuleNotFoundError: No module named 'svgling'

Tree(Lemma('state.v.01.say'), ['said'])